<a href="https://colab.research.google.com/github/Sangyups/VanillaGAN/blob/main/Vanilla_GAN(with_MNIST).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from __future__ import print_function
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
import torch.nn.functional as F
import time

# Training settings
batch_size = 64
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST Model on {device}\n{"=" * 44}')

# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


Training MNIST Model on cuda



Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw




Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw




Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [24]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.n_features = 128
        self.n_out = 784
        self.fc0 = nn.Sequential(
            nn.Linear(self.n_features, 256),
            nn.LeakyReLU(0.2)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2)
        )
        self.fc3 = nn.Sequential(
            nn.Linear(1024, self.n_out),
            nn.Tanh()
        )
    def forward(self, x):
        x = self.fc0(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.n_in = 784
        self.n_out = 1
        self.fc0 = nn.Sequential(
            nn.Linear(self.n_in, 1024),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.fc3 = nn.Sequential(
            nn.Linear(256, self.n_out),
            nn.Sigmoid()
        )
    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc0(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x


In [25]:
import torch

G = Generator().to(device)
D = Discriminator().to(device)

loss = torch.nn.BCELoss()

optimizer_G = optim.Adam(G.parameters(), lr=1e-4)
optimizer_D = optim.Adam(D.parameters(), lr=1e-4)


In [30]:
for epoch in range(10):
    for real_data, target in train_loader:
        real_data = real_data.to(device)
        real_label = torch.ones(real_data.shape[0], 1).to(device)
        fake_label = torch.zeros(real_data.shape[0], 1).to(device)
        
        noise = torch.randn(real_data.shape[0], 128).to(device)
        fake_data = G(noise)
        loss_g = loss(D(fake_data), real_label)
        optimizer_G.zero_grad()
        loss_g.backward()
        optimizer_G.step()

        fake_data = fake_data.detach()

        loss_d_real = loss(D(real_data), real_label)
        loss_d_fake = loss(D(fake_data), fake_label)
        loss_d_final = loss_d_real + loss_d_fake
        optimizer_D.zero_grad()
        loss_d_final.backward()
        optimizer_D.step()
    print("============epoch: ",epoch,"==========")
    print("Generator Loss:", loss_g.item())
    print("Discriminator Loss:", loss_d_final.item())

============epoch:  0 ==========
Generator Loss: 3.597993850708008
Discriminator Loss: 0.4080272912979126
============epoch:  1 ==========
Generator Loss: 3.6333167552948
Discriminator Loss: 0.541650652885437
============epoch:  2 ==========
Generator Loss: 4.1726579666137695
Discriminator Loss: 0.28564971685409546
============epoch:  3 ==========
Generator Loss: 4.33902645111084
Discriminator Loss: 0.20728528499603271
============epoch:  4 ==========
Generator Loss: 4.609706878662109
Discriminator Loss: 0.2331051379442215
============epoch:  5 ==========
Generator Loss: 4.3876824378967285
Discriminator Loss: 0.1447799801826477
============epoch:  6 ==========
Generator Loss: 4.596945762634277
Discriminator Loss: 0.2763339579105377
============epoch:  7 ==========
Generator Loss: 4.43265438079834
Discriminator Loss: 0.4558860659599304
============epoch:  8 ==========
Generator Loss: 4.812129974365234
Discriminator Loss: 0.24354392290115356
============epoch:  9 ==========
Generator Los